In [ ]:
#install
!pip install transformers
!pip install sklearn
!pip install tensorboardX
!pip install torch

In [2]:
cd /home/ec2-user/SageMaker/projects/transformers

/home/ec2-user/SageMaker/projects/transformers


In [3]:
!pip install -r ./examples/requirements.txt

You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
cd /home/ec2-user/SageMaker/projects/transformer_entity_rel/

/home/ec2-user/SageMaker/projects/transformer_entity_rel


In [17]:
import torch
from transformers import *
import run_glue_gap
import json
import pandas as pd
import numpy as np
import pickle


In [18]:
#filename='/home/ec2-user/SageMaker/Data/contexts_samples_df1.2.json'
#filename='/home/ec2-user/SageMaker/Data/contexts_samples_df1.json'
filename='/home/ec2-user/SageMaker/Data/contexts_samples_original1.json'

#dict_keys(['query_name', 'contexts', 'article_title', 'conservative-marked-body', 
    #'conservative-sequence', 'liberal-marked-body', 'entity_relevance', 'article_body', 'liberal-sequence'])
with open(filename, 'r') as f:
            the_dic = json.loads(f.read())


the_targets=[docca['entity_relevance'] for docca in the_dic]
the_inputs=[]
#NEED SOME BETTER WAY TO HANDLE LONGER SEQUENCES THAN JUST TAKING FIRST 250 TOKENS!
for docca in the_dic:
    the_inputs.append(" ".join(docca['sequence'].split()[:512]))
    #the_inputs.append(" ".join(docca['sequence'].split()[:250]))


print(len(the_targets))
print(len(the_inputs))
the_data = pd.DataFrame(list(zip(the_targets, the_inputs)), columns =['targets', 'texts']) 
print(the_data.head())
msk = np.random.rand(len(the_data)) < 0.9
train_data = the_data[msk]
test_data = the_data[~msk]
print(np.shape(test_data))
train_data.to_csv('/home/ec2-user/SageMaker/Data/entrel_data/train.tsv',sep='\t',index=False,header=False)
test_data.to_csv('/home/ec2-user/SageMaker/Data/entrel_data/dev.tsv',sep='\t',index=False,header=False)

3000
3000
  targets                                              texts
0       1  Glancy Prongay & Murray LLP Announces Investig...
1       1  The case has been assigned to Judge Keith P. E...
2       1  . It comes afterDavid Bolt, who provides indep...
3       1  ROSEN , A GLOBALLY RECOGNIZED LAW FIRM , Remin...
4       1  <BEGIN-SPAN> wells fargo wells fargo <END-SPAN...
(303, 2)


In [ ]:


import torch
from transformers import *
import run_glue_gap
import json
import pandas as pd
import numpy as np
import pickle

filename='/home/ec2-user/SageMaker/Data/contexts_samples_original4.json'

#dict_keys(['query_name', 'contexts', 'article_title', 'conservative-marked-body', 
    #'conservative-sequence', 'liberal-marked-body', 'entity_relevance', 'article_body', 'liberal-sequence'])
with open(filename, 'r') as f:
            the_dic = json.loads(f.read())

libandcon_cases=[]
libnotcon_cases=[]
atleastone_cases=[]
fail_cases=[]
#the_targets=[]
the_targets=[docca['entity_relevance'] for docca in the_dic]
conservative_inputs=[]
liberal_inputs=[]
for doc_ind, docca in enumerate(the_dic):
    con_body=docca['conservative-marked-body']
    con_seq=docca['conservative-sequence']
    lib_body=docca['liberal-marked-body']
    lib_seq=docca['liberal-sequence']
    if len(lib_seq)>0 and len(lib_body)>0: #at least lib_seq and lib_body not empty
        if len(lib_body.split())<len(lib_seq.split()):
            liberal_inputs.append(lib_body)
        else:
            liberal_inputs.append(lib_seq)
        if len(con_seq)>0 and len(con_body)>0: #con_seq and con_body also not empty
            libandcon_cases.append(doc_ind)
            if len(con_body.split())<len(con_seq.split()):
                conservative_inputs.append(con_body)
            else:
                conservative_inputs.append(con_seq)
        else: #lib not empty but con empty
            libnotcon_cases.append(doc_ind)
            conservative_inputs.append(liberal_inputs[-1])        
        #the_targets.append(docca['entity_relevance'])
    elif len(lib_seq)>0 or len(lib_body)>0 or len(con_seq)>0 or len(con_body)>0:
        atleastone_cases.append(doc_ind)
        if len(lib_body)>0:
            conservative_inputs.append(lib_body)
            liberal_inputs.append(lib_body)
        if len(lib_seq)>0:
            conservative_inputs.append(lib_seq)
            liberal_inputs.append(lib_seq)
        if len(con_body)>0:
            conservative_inputs.append(con_body)
            liberal_inputs.append(con_body)
        if len(con_seq)>0:
            conservative_inputs.append(con_seq)
            liberal_inputs.append(con_seq)
        #the_targets.append(docca['entity_relevance'])
    else:
        #the_targets.append('0')
        conservative_inputs.append('x')
        liberal_inputs.append('x')
        fail_cases.append(doc_ind)

print(len(libandcon_cases))
print(len(libnotcon_cases))
print(len(atleastone_cases))
print(fail_cases)
print(len(the_targets))
print(len(conservative_inputs))
print(len(liberal_inputs))
con_data = pd.DataFrame(list(zip(the_targets, conservative_inputs)), columns =['targets', 'texts']) 
lib_data = pd.DataFrame(list(zip(the_targets, liberal_inputs)), columns =['targets', 'texts']) 
con_data = con_data.sample(frac=1).reset_index(drop=True)#shuffle
lib_data = lib_data.sample(frac=1).reset_index(drop=True)#shuffle
print(con_data.head())
print(lib_data.head())
#con_data
msk = np.random.rand(len(con_data)) < 0.9
con_train_data = con_data[msk].reset_index(drop=True)
con_test_data = con_data[~msk].reset_index(drop=True)
print(con_test_data.head())
print(np.shape(con_test_data))
con_train_data.to_csv('/home/ec2-user/SageMaker/Data/entrel_data_con/train.tsv',sep='\t',index=False,header=False)
con_test_data.to_csv('/home/ec2-user/SageMaker/Data/entrel_data_con/dev.tsv',sep='\t',index=False,header=False)
#lib_data
msk = np.random.rand(len(lib_data)) < 0.9
lib_train_data = lib_data[msk].reset_index(drop=True)
lib_test_data = lib_data[~msk].reset_index(drop=True)
print(lib_test_data.head())
print(np.shape(lib_test_data))
lib_train_data.to_csv('/home/ec2-user/SageMaker/Data/entrel_data_lib/train.tsv',sep='\t',index=False,header=False)
lib_test_data.to_csv('/home/ec2-user/SageMaker/Data/entrel_data_lib/dev.tsv',sep='\t',index=False,header=False)

In [9]:
the_data.iloc[0]['texts']

'Glancy Prongay & Murray LLP Announces Investigation on Behalf of <BEGIN-SPAN> ternium <END-SPAN> S.A. Investors ( TX ). LOS ANGELES ( BUSINESS WIRE ) Glancy Prongay & Murray LLP ( “ GPM ” ) announces an investigation on behalf of <BEGIN-SPAN> ternium <END-SPAN> S.A. ( “ Tenaris ” or the “ Company ” ) ( NYSE. ternium TX ) investors concerning the Company and its officers’ possible violations of federal securities laws. ternium If you are a shareholder who suffered a loss, click here to participate. TX ) investors concerning the Company and its officers’ possible violations of federal securities laws ternium. If you are a shareholder who suffered a loss, click here to participate ternium. On November 27 , 2018 , Bloomberg published an article alleging that <BEGIN-SPAN> ternium <END-SPAN> ’ s Chairman Paolo Rocca , was indicted for his role in a graft scheme. ternium Specifically the article alleges that, “the judge charged Rocca after the Argentine billionaire testified that one of his 

In [11]:
the_dic[0].keys()

dict_keys(['article_body', 'contexts', 'sequence', 'article_title', 'query_name', 'entity_relevance'])

In [13]:
the_dic[0]['sequence']

'Glancy Prongay & Murray LLP Announces Investigation on Behalf of <BEGIN-SPAN> ternium <END-SPAN> S.A. Investors ( TX ). LOS ANGELES ( BUSINESS WIRE ) Glancy Prongay & Murray LLP ( “ GPM ” ) announces an investigation on behalf of <BEGIN-SPAN> ternium <END-SPAN> S.A. ( “ Tenaris ” or the “ Company ” ) ( NYSE. ternium  TX ) investors concerning the Company and its officers’ possible violations of federal securities laws. ternium   If you are a shareholder who suffered a loss, click here to participate. TX ) investors concerning the Company and its officers’ possible violations of federal securities laws   ternium. If you are a shareholder who suffered a loss, click here to participate  ternium. On November 27 , 2018 , Bloomberg published an article alleging that <BEGIN-SPAN> ternium <END-SPAN> ’ s Chairman Paolo Rocca , was indicted for his role in a graft scheme. ternium  Specifically the article alleges that, “the judge charged Rocca after the Argentine billionaire testified that one 

In [14]:
the_dic[0]['article_body']

'LOS ANGELES--(BUSINESS WIRE)--       Glancy      Prongay & Murray LLP  (“GPM”) announces an investigation on      behalf of Ternium S.A.   (“Tenaris” or the “Company”) (NYSE:  TX )      investors concerning the Company and its officers’ possible violations      of federal securities laws.    \n          If you are a shareholder who suffered a loss, click\xa0 here \xa0to      participate.    \n          On\xa0November 27, 2018,\xa0Bloomberg\xa0published an article alleging that      Ternium’s Chairman Paolo Rocca, was indicted for his role in a graft      scheme. Specifically the article alleges that, “the judge charged Rocca      after the Argentine billionaire testified that one of his company’s      executives paid an undisclosed amount of cash to government officials in      monthly installments from 2009 to 2012.” On this news, Ternium’s share      price fell $1.42 per share or nearly 5% to close at $28.02 per share on      November 27, 2018, thereby injuring investors.    \n     

In [ ]:

import torch
from transformers import *
import run_glue_gap
import json
import pandas as pd
import numpy as np
import pickle

parser = run_glue_gap.argparse.ArgumentParser()
args = run_glue_gap.get_args(parser)

datasetta=1 # 0: Gap, 1: entrel
model_typea=1 # 0: BERT, 1: XLNET, 2: Roberta, 3: Distilbert, 4: XLM, 5:TransformerXL
small_or_big=1 # 0: small, 1: big
batch_size=8

args.num_train_epochs=5.0
args.do_train=True
args.do_eval=True

#args.max_steps=3
args.overwrite_output_dir=True
args.overwrite_cache = True
args.evaluate_during_training=False
args.logging_steps =50
args.save_steps=1000
args.max_seq_length = 512

args.learning_rate =1e-5
args.adam_epsilon=1e-8
args.warmup_steps= 50  
args.weight_decay= 0.0

if small_or_big==0:
    args.per_gpu_eval_batch_size=8
    args.per_gpu_train_batch_size=8
    if batch_size==8:
        args.gradient_accumulation_steps=1
    elif batch_size==16:
        args.gradient_accumulation_steps=2
    elif batch_size==32:
        args.gradient_accumulation_steps=4
elif small_or_big==1:
    args.per_gpu_eval_batch_size=1
    args.per_gpu_train_batch_size=1
    if batch_size==8:
        args.gradient_accumulation_steps=8
    elif batch_size==16:
        args.gradient_accumulation_steps=16
    elif batch_size==32:
        args.gradient_accumulation_steps=32



if model_typea==0:
    args.model_type='bert'
    args.do_lower_case=True
    if small_or_big==0:
        args.model_name_or_path='bert-base-uncased'
    else:
        args.model_name_or_path='bert-large-uncased'
elif model_typea==1:
    args.model_type='xlnet'
    args.do_lower_case=False
    #args.model_name_or_path='xlnet-base-cased'
    if small_or_big==0:
        args.model_name_or_path='xlnet-base-cased'
    else:
        args.model_name_or_path='xlnet-large-cased'
elif model_typea==2:
    args.model_type='roberta'
    args.do_lower_case=False
    #args.model_name_or_path='roberta-base'
    if small_or_big==0:
        args.model_name_or_path='roberta-base'
    else:
        args.model_name_or_path='roberta-large'
elif model_typea==3:
    args.model_type='distilbert'
    args.model_name_or_path='distilbert-base-uncased'
    args.do_lower_case=True
elif model_typea==4:
    args.model_type='xlm'
    args.model_name_or_path='xlm-mlm-en-2048'
    args.do_lower_case=False

if datasetta==0:
    args.data_dir='/home/ec2-user/SageMaker/Data/gap_data/'
    args.output_dir='/tmp/gap/'
    args.task_name='gap'
elif datasetta==1:
    args.data_dir='/home/ec2-user/SageMaker/Data/entrel_data'
    args.output_dir='/tmp/entrel/'
    args.task_name='entityrel'
    

#args.per_gpu_eval_batch_size=2
#args.per_gpu_train_batch_size=2
#args.gradient_accumulation_steps=16

results=run_glue_gap.run_main(parser,args)


In [ ]:


import torch
from transformers import *
import run_glue_gap
import json
import pandas as pd
import numpy as np
import pickle


args_list=[]
results_list=[]

num_trials=1000

for trial_num in range(num_trials):
    
    parser = run_glue_gap.argparse.ArgumentParser()
    args = run_glue_gap.get_args(parser)
    
    #model_typea=0 # 0: BERT, 1: XLNET, 2: Roberta, 3: Distilbert
    model_typea=run_glue_gap.gen_grid_val([0,1,2],'sel')
    small_or_big=run_glue_gap.gen_grid_val([0,1],'sel') # 0: small, 1: big
    args.con_or_lib=run_glue_gap.gen_grid_val([0,1],'sel') # 0: con, 1: lib
    batch_size=run_glue_gap.gen_grid_val([8,16,32],'sel')
    
    args.num_train_epochs=run_glue_gap.gen_grid_val([5,8,11],'sel')
    #args.num_train_epochs=run_glue_gap.gen_grid_val([1,2],'sel')
    args.do_train=True
    args.do_eval=True
    
    #args.max_steps=3
    args.overwrite_output_dir=True
    args.overwrite_cache = True
    args.evaluate_during_training=False
    args.logging_steps =50
    args.save_steps=1000
    args.max_seq_length = 512
    
    args.learning_rate =run_glue_gap.gen_grid_val([1e-6,5e-5],'exp')
    args.adam_epsilon= 1e-8
    args.warmup_steps= run_glue_gap.gen_grid_val([0,100],'lin_round')
    args.weight_decay= 0.0
    
    if small_or_big==0:
        args.per_gpu_eval_batch_size=8
        args.per_gpu_train_batch_size=8
        if batch_size==8:
            args.gradient_accumulation_steps=1
        elif batch_size==16:
            args.gradient_accumulation_steps=2
        elif batch_size==32:
            args.gradient_accumulation_steps=4
    elif small_or_big==1:
        args.per_gpu_eval_batch_size=1
        args.per_gpu_train_batch_size=1
        if batch_size==8:
            args.gradient_accumulation_steps=8
        elif batch_size==16:
            args.gradient_accumulation_steps=16
        elif batch_size==32:
            args.gradient_accumulation_steps=32
    
    if model_typea==0:
        args.model_type='bert'
        args.do_lower_case=True
        if small_or_big==0:
            args.model_name_or_path='bert-base-uncased'
        else:
            args.model_name_or_path='bert-large-uncased'
    elif model_typea==1:
        args.model_type='xlnet'
        args.do_lower_case=False
        #args.model_name_or_path='xlnet-base-cased'
        if small_or_big==0:
            args.model_name_or_path='xlnet-base-cased'
        else:
            args.model_name_or_path='xlnet-large-cased'
    elif model_typea==2:
        args.model_type='roberta'
        args.do_lower_case=False
        #args.model_name_or_path='roberta-base'
        if small_or_big==0:
            args.model_name_or_path='roberta-base'
        else:
            args.model_name_or_path='roberta-large'
    elif model_typea==3:
        args.model_type='distilbert'
        args.model_name_or_path='distilbert-base-uncased'
        args.do_lower_case=True
    elif model_typea==4:
        args.model_type='xlm'
        args.model_name_or_path='xlm-mlm-en-2048'
        args.do_lower_case=False
    
    if args.con_or_lib==0:
        args.data_dir='/home/ec2-user/SageMaker/Data/entrel_data_con'
    else:
        args.data_dir='/home/ec2-user/SageMaker/Data/entrel_data_lib'
    args.output_dir='/tmp/entrel/'
    args.task_name='entityrel'
    
    results=run_glue_gap.run_main(parser,args)
    
    args_list.append(args)
    results_list.append(results)
    
    outputta=(results_list,args_list)
    pickle.dump(outputta, open("outputta.pickle","wb"))

